In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Vocab import *
from model import AttentionModel

import pandas as pd
import os
import string
import random

In [2]:
#Hyperparams
lr = 1
embed_size = 256
hidden_size = 512

In [3]:
vocab = getVocab('text.txt')

In [4]:
vocab_size = vocab.size()

In [5]:
model = AttentionModel(embed_size, hidden_size, vocab_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [6]:
corr_sentences = pd.read_csv("data/qs.csv")["Correct Answer"]
corr_sentences = [cleanText(sentence) for sentence in corr_sentences]
corr_tensors = [torch.Tensor(vocab.getSentenceArray(sentence)) for sentence in corr_sentences]

In [7]:
test_sentences = pd.read_csv("data/ratings.csv")
test_sentences["Answer"] = [cleanText(s) for s in test_sentences["Answer"]]
test_tensors = [torch.tensor(vocab.getSentenceArray(s)) for s in test_sentences["Answer"]]

In [8]:
for e in range(100):
    total_loss = 0
    for t in range(len(test_tensors)):
        curr_a = random.randint(0, len(test_tensors)-1) 
        curr_q = test_sentences["Question Id"][curr_a]
        
        optimizer.zero_grad()
        
        pred = model(corr_tensors[curr_q].long(), test_tensors[curr_a].long()).view(1)
        y = torch.tensor(test_sentences["Rating"][curr_a]).float().view(1)
        loss = criterion(pred, y)
        total_loss += loss.item()
    
        loss.backward()
        optimizer.step()
    print("Epoch:", e, "Loss:", total_loss)

/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 0 Loss: 1.255399560672231
Epoch: 1 Loss: 1.6752484247554094
Epoch: 2 Loss: 0.7223923996116355
Epoch: 3 Loss: 0.27907300597507856
Epoch: 4 Loss: 0.3705850966960611
Epoch: 5 Loss: 0.1973770975764637
Epoch: 6 Loss: 0.11212126887403429
Epoch: 7 Loss: 0.21689261182473274
Epoch: 8 Loss: 0.1864827438375869
Epoch: 9 Loss: 0.07967185523011722
Epoch: 10 Loss: 0.059420701960334554
Epoch: 11 Loss: 0.3342405492439866
Epoch: 12 Loss: 0.28216697729658335
Epoch: 13 Loss: 0.14254923211410642
Epoch: 14 Loss: 0.10091169067891315
Epoch: 15 Loss: 0.08749430440366268
Epoch: 16 Loss: 0.11342332896310836
Epoch: 17 Loss: 0.09209841064875945
Epoch: 18 Loss: 0.0609575970484002
Epoch: 19 Loss: 0.04224176568095572
Epoch: 20 Loss: 0.06830393372183607
Epoch: 21 Loss: 0.10392664890969172
Epoch: 22 Loss: 0.037127156829228625
Epoch: 23 Loss: 0.2288047257134167
Epoch: 24 Loss: 0.1183038246090291
Epoch: 25 Loss: 0.0377021835593041
Epoch: 26 Loss: 0.06317808679392556
Epoch: 27 Loss: 0.026460274983492127
Epoch: 28 L

In [9]:
torch.save(model, "saved_models/model2.pt")